<a href="https://colab.research.google.com/github/tianhaoz95/ultron/blob/dev%2Ftry-atari-game/notebooks/prototype_atari_with_actor_critic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg x11-utils > /dev/null 2>&1

In [0]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (46.1.3)


In [0]:
import gym
import threading
import math
import glob
import io
import base64
import matplotlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from gym.wrappers import Monitor
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:
class Worker(threading.Thread):
  # The cumulative episode played
  global_eps = 0

  def __init__(self):
    super(Worker, self).__init__()
    print('not implemented')

  def compute_loss(self):
    print('not implemented')

  def compute_gradient(self):
    print('not implemented')
  
  def run(self, args):
    # Prepare variables
    self.args = args

    # Check the maximum episode of learning is reached
    while Worker.global_eps < self.args.max_eps:
      done = False
      # Check if the game is over
      while not done:
        # Get the policy and play the game

        # If done (game is over), calculate loss
        if (done):
          # Compute gradient and update models
          with tf.GradientTape() as tape:
            # Compute gradient
            loss = self.compute_loss()
            grads = self.compute_gradient()
            # Update models

In [0]:
class ActorCriticModel():
  def __init__(self):
    print('not implemented')

In [0]:
class BaselineModel():
  def __init__(self):
    print('not implemented')

In [0]:
class MasterAgent():
  def __init__(self):
    print('not implemented')

In [0]:
def gym_sanity_check():
  print('starting gym environment sanity check')
  env = wrap_env(gym.make('Breakout-ram-v0'))
  for _ in range(20):
      observation = env.reset()
      done = False
      while not done:
          env.render()
          action = env.action_space.sample()
          observation, _, done, _ = env.step(action)
  env.close()
  show_video()

In [0]:
def entry_point(mode='train'):
  if mode == 'sanity':
    gym_sanity_check()
  print('Hello World')

In [0]:
entry_point(mode='sanity')

starting gym environment sanity check


Hello World
